In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

lokalizacjafunkcjonalna = pd.read_csv("csv_files/lokalizacja_funkcjonalna.csv", encoding="utf-8-sig", sep=";")

lokalizacjafunkcjonalna = lokalizacjafunkcjonalna[["Lokaliz. funkc.", "Oznaczenie"]]

lokalizacjafunkcjonalna.columns = ["LokalizacjaFunkcjonalnaId", "LokalizacjaFunkcjonalnaNazwa"]

lokalizacjafunkcjonalna["LiniaId"] = lokalizacjafunkcjonalna["LokalizacjaFunkcjonalnaId"].str.extract(r'^[^-]+-[^-]+-[^-]+-([0-9]+)(?:-|$)')
lokalizacjafunkcjonalna["LiniaId"] = pd.to_numeric(lokalizacjafunkcjonalna["LiniaId"], errors='coerce')

lokalizacjafunkcjonalna = lokalizacjafunkcjonalna.dropna(subset=["LokalizacjaFunkcjonalnaId"])

lokalizacjafunkcjonalna = lokalizacjafunkcjonalna.drop_duplicates(subset=["LokalizacjaFunkcjonalnaId"])

lokalizacjafunkcjonalna["LiniaId"] = lokalizacjafunkcjonalna["LokalizacjaFunkcjonalnaId"].str.extract(r'^(?:[^-]+-){3}([0-9]+)')
lokalizacjafunkcjonalna["LiniaId"] = pd.to_numeric(lokalizacjafunkcjonalna["LiniaId"], errors='coerce')

lokalizacjafunkcjonalna["UGP"] = lokalizacjafunkcjonalna["LokalizacjaFunkcjonalnaId"].str.extract(r'(?:PLPA-)?([A-Z]{2}-[A-Z0-9]{3})')
lokalizacjafunkcjonalna["UGP"] = lokalizacjafunkcjonalna["UGP"].astype(str)

server = 'JASNACZERN'        
database = 'ferrero'
username = 'sa'
password = '1234'
driver = 'ODBC Driver 17 for SQL Server'

conn_str = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)
engine = create_engine(conn_str)

existing_ids = pd.read_sql("SELECT LokalizacjaFunkcjonalnaId FROM LokalizacjaFunkcjonalna", engine)
lokalizacjafunkcjonalna = lokalizacjafunkcjonalna[~lokalizacjafunkcjonalna["LokalizacjaFunkcjonalnaId"].isin(existing_ids["LokalizacjaFunkcjonalnaId"])]

try:
    lokalizacjafunkcjonalna.to_sql("lokalizacjafunkcjonalna", con=engine, if_exists="append", index=False)
    print("Data inserted into lokalizacjafunkcjonalna table.")
except Exception as e:
    print("Insert failed:", e)

Data inserted into lokalizacjafunkcjonalna table.
